__Step 1__Model-based feature selection (SelectFromModel)

        https://www.kaggle.com/residentmario/automated-feature-selection-with-sklearn



   __Step 1.1__Estimating Regression Model and assessing overall model fit.

    This stage includes:

    -- Select method for specifying the Regression Model. This can be done using:

        Stepwise Regression:--  
        Forward Selection - Backward Elimination
    
        Instead of manually selecting the variables, we can automate this process by using forward or backward selection. 
        Forward selection starts with most significant predictor in the model and adds variable for each step. Backward 
        elimination starts with all predictors in the model and removes the least significant variable for each step. 
    
__Step 2__Examine the statistical significance of the model by checking:

 -- Selecting criteria can be set to any statistical measure like R-square, t-stat etc.

 -- anova(analysis of variance) -(https://pythonfordatascience.org/anova-python/)

 -- F-Statistic -  An F test tests a model which may have multiple explanatory variables contributing to the variance explained 
     by the model.

 -- Coefficient of determination - R2 & Adjusted R2

 -- Residual Standard Error /Standard error of estimate (Se)
    https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/
    
 -- Statistical significance of the estimated parameters using t-test

__Step 3__Test the Regression Variable for meeting regression assumptions.

 -- Use residual plots to test the assumptions of: linearity, homoscedasticity, independence and normality.
    https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/

__Step 4__Perform Enhanced Regression diagnostics 

- Run the omnibus test gvlma(fit) that performs a global validation of linear model assumptions 

__Step 5__Identify the Influential Observations

In [22]:
%store -r orders_ML
%store -r st_test_set
%store -r st_train_set

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import numpy as np
import scipy.stats as st
from sklearn.feature_extraction.text import CountVectorizer
from IPython.core.display import Image, display
from IPython.core.display import HTML
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error


# Machine learning library/functions
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
np.random.seed(42)
from sklearn import linear_model
import sklearn.linear_model
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# Additional librariers used for modeling and evaluation
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.model_selection import cross_val_score

In [37]:
st_train_set.shape

(80000, 18)

In [38]:
st_test_set.shape

(20000, 18)

In [53]:
train_X =st_train_set.drop(['review_score'],axis=1).values

train_y = st_train_set['review_score'].values 

test_X= st_test_set.drop(['review_score'],axis=1).values
test_y = st_test_set['review_score'].values 

In [54]:
reg = GradientBoostingRegressor()
reg.fit(train_X, train_y)
reg.score(test_X, test_y)

0.26554034056331344

In [72]:
search_space = [{"classifier":LinearRegression(),
                 'params':{'n_jobs':[-1]}},
                {"classifier":RandomForestRegressor(n_jobs=-1),
                 "params":{"n_estimators": np.arange(8,9,1),"max_features":['sqrt', 'log2']}},
               {"classifier": GradientBoostingRegressor(),
                'params':{"alpha": np.arange(0.7,0.9,.1), "loss":['ls','quantile']}},
               ]
best_acc = 0.0
best_clf = 0
for row in search_space:
    random_search = GridSearchCV(row['classifier'],param_grid = row['params'], cv=5)
    random_search.fit(train_X, train_y.ravel())
    print (random_search.best_params_)
    predictions = random_search.predict(test_X)
    accuracy = metrics.r2_score(test_y, predictions)
    mse = metrics.mean_squared_error(test_y, predictions)
    print("mean squared error:", mse)
    print('Test set accuracy (r2_score) for best params: %.3f ' % accuracy)
    #x_plot = plt.scatter(predictions, (predictions - test_y), c='b')
    #plt.hlines(y=0, xmin= -1000, xmax=5000)
    #plt.title('Residual plot')
    if accuracy > best_acc:
          best_acc = accuracy
          best_clf = row['classifier']
print("Best Acuracy score: %.3f " %best_acc)
print("Best Regressor: ",best_clf)

{'n_jobs': -1}
mean squared error: 0.7952038634524773
Test set accuracy (r2_score) for best params: 0.205 
{'max_features': 'log2', 'n_estimators': 8}
mean squared error: 0.8212548108370044
Test set accuracy (r2_score) for best params: 0.179 
{'alpha': 0.8999999999999999, 'loss': 'ls'}
mean squared error: 0.7344294002312267
Test set accuracy (r2_score) for best params: 0.266 
Best Acuracy score: 0.266 
Best Regressor:  GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


In [57]:
lm = sklearn.linear_model.LinearRegression()
model = lm.fit(train_X, train_y)
predictions = lm.predict(test_X)
#print(lm.score(X2_train,y2_train))
print(lm.score(test_X,test_y))

0.20477717706368637


In [58]:
prediction_df = pd.DataFrame(predictions, columns=['predictions'])
actual_df = pd.DataFrame(test_y, columns=['actual'])
actual_dfx2 = pd.DataFrame(test_X)

X2_final = pd.concat([actual_dfx2, prediction_df, actual_df],axis=1)
X2_final

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,predictions,actual
0,0.0,11.0,-0.499285,-0.380282,-0.217973,-0.199894,-2.449841,-0.091707,-0.158960,-0.514632,-0.392834,0.944162,0.095470,-0.712011,-0.282071,-0.504091,0.482252,0.197210,0.693679
1,0.0,4.0,-0.380791,-0.022859,-0.217973,-0.199894,-0.773194,0.571067,0.984229,-0.366173,0.115163,0.620059,0.095470,-0.437678,-0.282071,-0.373392,0.294165,0.161121,0.693679
2,0.0,2.0,-0.061026,-0.363854,-0.217973,-0.199894,0.410322,-0.314135,-0.730555,-0.094170,-1.408829,-0.838404,-0.655716,0.110988,-0.282071,-0.020690,-0.608085,0.259094,-0.039682
3,0.0,2.0,-0.355594,-0.617236,-0.217973,-0.199894,0.903453,0.584593,-0.730555,-0.400652,-0.392834,-1.324558,-0.805954,0.385322,-0.282071,-0.345599,-0.495600,0.269903,0.693679
4,0.0,11.0,-0.458230,-0.359374,-0.217973,-0.199894,0.410322,-0.954367,-0.158960,-0.473336,-1.408829,0.620059,-0.355242,-0.849178,-0.282071,-0.458808,0.215310,0.220897,0.693679
5,0.0,2.0,-0.457717,1.849876,-0.217973,-0.199894,0.311696,-0.841650,-0.730555,-0.255185,0.623160,-1.486610,-0.655716,0.659655,-0.282071,-0.458242,4.004595,0.218722,-0.039682
6,0.0,2.0,-0.000984,-0.411146,-0.217973,-0.199894,-0.378688,-0.015060,-0.730555,-0.041447,1.131158,-0.514301,-0.205005,0.248155,-0.282071,0.045537,-0.671395,0.178674,-0.039682
7,0.0,2.0,-0.596328,-0.694893,-0.217973,-0.199894,0.508948,-0.258528,-0.730555,-0.638372,0.115163,-1.324558,-0.805954,0.385322,-0.282071,-0.611130,1.018609,0.258672,0.693679
8,4.0,2.0,0.044022,2.244136,-0.217973,-0.199894,0.804827,-0.698875,-0.730555,0.263170,0.623160,-0.676352,-1.256665,-0.574845,-0.282071,0.095179,0.629248,-2.714189,-2.239766
9,0.0,1.0,-0.192042,-0.599813,-0.217973,1.210890,1.100706,-0.339684,-0.730555,-0.242629,1.639155,0.133905,0.395944,0.248155,-0.282071,-0.165201,-0.667656,-0.174484,-2.239766
